# Week 3: AI Tutor with Synthetic Data Generator

Uses Hugging Face Hub (pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig) to generate synthetic teaching scenarios. The tutor inserts these into responses.

In [ ]:
!pip install -q transformers accelerate bitsandbytes gradio python-dotenv openai

In [ ]:
import os
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch

try:
    from google.colab import userdata
    HF_TOKEN = userdata.get("HF_TOKEN")
except Exception:
    load_dotenv(override=True)
    HF_TOKEN = os.getenv("HF_TOKEN")

from huggingface_hub import login
if HF_TOKEN:
    login(token=HF_TOKEN)
    print("HF login OK")
else:
    print("Set HF_TOKEN in Colab Secrets or .env")

In [ ]:
# Data generator model (Colab-friendly: small + 4-bit)
GEN_MODEL = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
MAX_GEN_TOKENS = 256

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
)

tokenizer = AutoTokenizer.from_pretrained(GEN_MODEL)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    GEN_MODEL,
    device_map="auto",
    quantization_config=quant_config,
)

gen_pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=MAX_GEN_TOKENS,
)

In [ ]:
def generate_synthetic_examples(topic: str) -> str:
    """Generate 1–2 short teaching scenarios/examples for the given topic."""
    prompt = f"Generate two very short teaching scenarios or concrete code examples for explaining: {topic}. Each in 1-2 sentences. No preamble."
    out = gen_pipe(prompt, do_sample=True, temperature=0.7)
    text = out[0]["generated_text"] if out else ""
    return text.replace(prompt, "").strip()[:500] if text else ""

In [ ]:
# OpenRouter: in Colab use Secrets (🔑 sidebar); locally use .env or env var
try:
    from google.colab import userdata
    IN_COLAB = True
    api_key = userdata.get("OPENROUTER_API_KEY")
except Exception:
    IN_COLAB = False
    load_dotenv(override=True)
    api_key = os.getenv("OPENROUTER_API_KEY")

BASE_URL = "https://openrouter.ai/api/v1"
openrouter = OpenAI(api_key=api_key, base_url=BASE_URL) if api_key else None
OLLAMA_BASE_URL = "http://localhost:11434/v1"
ollama = OpenAI(base_url=OLLAMA_BASE_URL, api_key="ollama")

# In Colab only OpenRouter is available (Ollama runs locally)
MODEL_CHOICES = [
    ("gpt-4o-mini (OpenRouter)", "gpt-4o-mini", "openrouter"),
    ("llama3.2 (Ollama)", "llama3.2", "ollama"),
] if not IN_COLAB else [
    ("gpt-4o-mini (OpenRouter)", "gpt-4o-mini", "openrouter"),
]

BASE_SYSTEM = """You are a professional AI coding tutor. Give clear, step-by-step explanations with code examples. Use the following synthetic teaching scenarios or examples to enrich your answer when relevant. Weave them into your explanation.

Synthetic scenarios/examples to use when helpful:
{synthetic}

Keep a friendly, expert tone. Respond in markdown (e.g. code blocks)."""

In [ ]:
def stream_reply(history, user_message, model_label, use_synthetic):
    label_to = {l: (mid, back) for l, mid, back in MODEL_CHOICES}
    model_id, backend = label_to.get(model_label, MODEL_CHOICES[0][1:])
    client = openrouter if backend == "openrouter" else ollama
    if not client:
        if backend == "openrouter":
            if IN_COLAB:
                yield (
                    "**OpenRouter** key not found. In Colab: open the **Secrets** panel (key icon in left sidebar)", 
                )
            else:
                yield (
                    "**OpenRouter** is not set. Add `OPENROUTER_API_KEY` to .env, "
                )
        else:
            yield "Ollama is for local runs only. In Colab use OpenRouter."
        return
    synthetic = generate_synthetic_examples(user_message[:200]) if use_synthetic else "(none)"
    system = BASE_SYSTEM.format(synthetic=synthetic)
    messages = [{"role": "system", "content": system}]
    for u, a in history:
        messages += [{"role": "user", "content": u}, {"role": "assistant", "content": a or ""}]
    messages.append({"role": "user", "content": user_message})
    stream = client.chat.completions.create(model=model_id, messages=messages, stream=True)
    for chunk in stream:
        part = (chunk.choices[0].delta.content or "")
        if part:
            yield part

def chat(message, history, model_choice, use_synthetic):
    if not (message or message.strip()):
        return
    full = ""
    for chunk in stream_reply(history, message, model_choice, use_synthetic):
        full += chunk
        yield full

In [ ]:
model_dropdown = gr.Dropdown(choices=[l for l, _, _ in MODEL_CHOICES], value=MODEL_CHOICES[0][0], label="Model")
use_synthetic = gr.Checkbox(value=True, label="Inject synthetic teaching scenarios")
demo = gr.ChatInterface(
    chat,
    additional_inputs=[model_dropdown, use_synthetic],
    title="Technical Q&A Tutor + Synthetic Data",
    description="Ask a coding question. Toggle to inject HF-generated scenarios into the answer.",
)
demo.launch()